In [ ]:
#import packages
import pandas as pd
import numpy as np
import datetime
import scipy
from scipy import stats
from scipy.optimize import curve_fit

In [ ]:
#I#SOTOPE DATA##
#read in isotope data and turn it into pandas dataframe
SG_water=pd.read_csv('/path/to/file/SGiso.csv')
df=pd.DataFrame(SG_water)
sns.set_theme(style="white")

#set timestamp, calculate d-excess, group data by sample type, streams and storm number
df['timestamp']=df['Date']+' '+df['Time']
df['date_time']=pd.to_datetime(df['timestamp'])
df['d-excess']=df['𝛿D']-8*df['𝛿18O']
streams=df.loc[df['Sample Type'] == 'Stream']
rain=df.loc[df['Sample Type'] == 'Rain']
intrain=df.loc[df['Sample Type'] == 'Integrated Rain']
storm1=df.loc[df['Storm'] =='storm 1']
storm2=df.loc[df['Storm'] =='storm 2']
storm3=df.loc[df['Storm'] =='storm 3']
storm4=df.loc[df['Storm'] =='storm 4']
storm5=df.loc[df['Storm'] =='storm 5']
storm6=df.loc[df['Storm'] =='storm 6']
storm7=df.loc[df['Storm'] =='storm 7']
df_sorted=df.sort_values(by='date_time')

#make custom color map by location
colors=['#F26513','#F2921D', '#019966' ]
locations=['Louise','Thelma', 'Henry']
cmap=dict(zip(locations, colors))

In [ ]:
##RAINFALL DATA##
#read in rainfall data and turn it into pandas dataframe
precip=pd.read_csv('/path/to/data/Bobat_Fire_Rain_Gauge_Louise.csv')
precip=pd.DataFrame(precip)

#set date time and other titles
precip['date_time']=pd.to_datetime(precip['Date and Time'])
precip['rain_units']=precip['Rainfall units']
precip['cum_rain_cm']=precip['Cumulative Rainfall  (cm) ']
precip['rain_cm']=precip['Rainfall (cm)']

In [ ]:
#calculate average precipitation weighted isotopic value for each storm
storm1['weightediso']=(storm1['𝛿18O']*storm1['Rain (cm)'])/(storm1['Rain (cm)'].sum())
s1pw18O=storm1['weightediso'].sum()
storm2['weightediso']=(storm2['𝛿18O']*storm2['Rain (cm)'])/(storm2['Rain (cm)'].sum())
s2pw18O=storm2['weightediso'].sum()
storm3['weightediso']=(storm3['𝛿18O']*storm3['Rain (cm)'])/(storm3['Rain (cm)'].sum())
s3pw18O=storm3['weightediso'].sum()
storm4['weightediso']=(storm4['𝛿18O']*storm4['Rain (cm)'])/(storm4['Rain (cm)'].sum())
s4pw18O=storm4['weightediso'].sum()
storm5['weightediso']=(storm5['𝛿18O']*storm5['Rain (cm)'])/(storm5['Rain (cm)'].sum())
s5pw18O=storm5['weightediso'].sum()
storm6['weightediso']=(storm6['𝛿18O']*storm6['Rain (cm)'])/(storm6['Rain (cm)'].sum())
s6pw18O=storm6['weightediso'].sum()
storm7['weightediso']=(storm7['𝛿18O']*storm7['Rain (cm)'])/(storm7['Rain (cm)'].sum())
s7pw18O=storm7['weightediso'].sum()

In [ ]:
##Calculate relationship between rainfall intensity, Fnew and Kfs values##

In [ ]:
##Fnew## 
#resample rainfall data for every 30 minutes, change units to mm/hr and find log rainfall
precip_30=pd.DataFrame()
precip_30['rain_cm']=precip.rain_cm.resample('30min', closed="left", label='right', origin='start').sum()
precip_30['rain_mm']=precip_30['rain_cm']*10
precip_30['rain_mmhr']=precip_30['rain_mm']*4
precip_30['rain_mmhr']=precip_30['rain_mmhr'].replace(0,np.nan)
precip_30['log_rain_mmhr']=  np.log10(precip_30['rain_mmhr'])


In [ ]:
##read in fnew isotope data, add timstamp and locate storms with Fnew values ##
fnew=pd.read_csv('/path/to/data/AtwoodHille_SanGabes_Fnew_isotopes.csv')
fnew['timestamp']=fnew['Date']+' '+fnew['Time']
fnew['date_time']=pd.to_datetime(fnew['timestamp'])
fnew=fnew.set_index('date_time')
storm4_fnew=fnew.loc['2021-12-14':'2021-12-15']
storm4_fnew=storm4_fnew.sort_values(by='date_time')
storm5_fnew=fnew.loc['2021-12-23':'2021-12-24']
storm5_fnew=storm5_fnew.sort_values(by='date_time')

In [ ]:
##Kfs Values ##
#read in Kfs values
SG_Ks=pd.read_csv('/path/to/data/Ks_results.csv')
Ks=pd.DataFrame(SG_Ks)

##replace 0 infiltration with extremely low infiltration rate for log conversion 
Ks.replace(0,0.000001,inplace=True)

#drop values higher than 3 standard deviation based on z-score
z = np.abs(stats.zscore(Ks['Ks (mm/hr)']))
print(np.where(z>3))
Ks=Ks.drop([25])
#calculate log of Ks values
Ks['log_ks_mmhr']= np.log10(Ks['Ks (mm/hr)'])
z = np.abs(stats.zscore(Ks['log_ks_mmhr']))
print(np.where(z > 3))

#separate data by catchment
henry=Ks.loc[Ks['Location']== 'Henry']
louise=Ks.loc[Ks['Location']== 'Louise']
thelma=Ks.loc[Ks['Location']== 'Thelma']
#find median log kfs values in mm/hr
HKfs=henry['log_ks_mmhr'].median()
LKfs=louise['log_ks_mmhr'].median()

#calculate the 30 min precipitation intensity devided by the median log kfs value at each catchment
precip_30['rain_intens/Lkfs']= precip_30['log_rain_mmhr']/LKfs
precip_30['rain_intens/Hkfs']= precip_30['log_rain_mmhr']/HKfs

#locate 30minute precip data for storms with calculable Fnew values  
storm4_precip=precip_30.loc['2021-12-14':'2021-12-15']
storm5_precip=precip_30.loc['2021-12-23':'2021-12-24']

In [ ]:
#merge storm precipitation and storm fnew value based on date time
#for storm 4
merged=pd.merge_asof(storm4_fnew, storm4_precip, on="date_time", direction="nearest")
louise_merged=merged.loc[merged['Location'] =='Louise']
henry_merged=merged.loc[merged['Location'] =='Henry']
#for storm 5
merged2=pd.merge_asof(storm5_fnew, storm5_precip, on="date_time", direction="nearest")
louise_merged2=merged2.loc[merged2['Location'] =='Louise']
henry_merged2=merged2.loc[merged2['Location'] =='Henry']
#drop any non number values from the four datasets
henry_merged2=henry_merged2.dropna()
louise_merged2=louise_merged2.dropna()
henry_merged=henry_merged.dropna()
louise_merged=louise_merged.dropna()
#combine locational datasets
louise=pd.concat([louise_merged,  louise_merged2])
henry=pd.concat([henry_merged, henry_merged2])
#drop outliers
merged=pd.concat([louise, henry])
z = np.abs(stats.zscore(merged['Fnew']))
print(np.where(z > 3))
merged.drop(merged.index[[37, 48, 49, 52]], inplace=True)

#separate datasets by location after droping outliers
henry_merged=merged.loc[merged['Location'] =='Henry']
louise_merged=merged.loc[merged['Location'] =='Louise']
louise_merged['Kfs_reg']=louise_merged['rain_intens/Lkfs']
henry_merged['Kfs_reg']=henry_merged['rain_intens/Hkfs']
merged=pd.concat([louise_merged, henry_merged])


In [ ]:
# calculate function to fit for linear regression:
def fit_func(x, m, b):
    return (m*x) + b

In [ ]:
# calculate linear regression of rainfall intenisty/kfs vs Fnew
# model fit on full sample of data:
Fpars, Fcov = curve_fit(f=fit_func, xdata=merged['Kfs_reg'], ydata=merged['Fnew']) 
# Get the axis limits
left, right = ax.get_xlim()
bottom, top = ax.get_ylim()

# get x axis limits
xlim = plt.xlim()
# fitted plot values (for CI)
x_fitted = (merged['Kfs_reg'])
y_fitted = fit_func(x_fitted, *Fpars)

residuals=(merged['Fnew']-y_fitted)
ss_res=np.sum(residuals**2)
ss_tot=np.sum((merged['Fnew']-np.mean(merged['Fnew']))**2)
r_squared=1-(ss_res/ss_tot)
print(r_squared)